In [1]:
from src.data_loader import DatabaseConnection

In [2]:
db_connection = DatabaseConnection(
    ssh_host='www.redesign777.tokyo',
    ssh_port=22,
    ssh_user='rddev',
    ssh_password='rdDev@20240521',
    db_host='localhost',
    db_port=3306,
    db_user='rddev',
    db_password='rdSql@20240521',
    db_name='redesign'
)
db_connection.start_tunnel()

2024-08-08 10:36:04,884| ERROR   | Password is required for key C:\Users\yshim/.ssh\id_rsa


SSHトンネルが 58058 にバインドされました。


In [3]:
# クエリを実行
df_frame = db_connection.execute_query("SELECT * FROM re_m_frame")
df_katashiki = db_connection.execute_query("SELECT * FROM re_m_katashiki")
df_pachinko = db_connection.execute_query("SELECT * FROM re_m_katashikidetailpachinko")
df_slot = db_connection.execute_query("SELECT * FROM re_m_katashikidetailslot")
df_katashiki_frame_relation = db_connection.execute_query("SELECT * FROM re_m_katashikiframerelation")
df_maker = db_connection.execute_query("SELECT * FROM re_m_maker")
df_model_evaluation = db_connection.execute_query("SELECT * FROM re_m_modelevaluation")
df_pachinko_type = db_connection.execute_query("SELECT * FROM re_m_pachinkotype")
df_pachinko_type_detail = db_connection.execute_query("SELECT * FROM re_m_pachinkotypedetail")
# SSHトンネルを閉じる
db_connection.close_tunnel()

SSHトンネルを閉じました。


In [4]:
def get_actual_pred_abs_diff(df, en_jp_dict):
    for target in targets:
        df[en_jp_dict[target] + '_絶対差分'] = df[target] - df[target + '_pred']
        df[en_jp_dict[target] + '_絶対差分'] = df[en_jp_dict[target] + '_絶対差分'].abs()
    return df

In [5]:
from data_columns.en_jp_dictionary import *

features_en_to_jp = {}
features_en_to_jp.update(features_jp)
features_en_to_jp.update(targets_jp)
features_en_to_jp.update(targets_pred_jp)

# 推論20231023～20240621

In [6]:
from src.data_preprocessing import data_preprocessing_operator
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

# start_date :　抽出データ開始日
# end_date : 抽出データ終了日
# openai_api_key : openaiのAPIキー
# use_existing_brand : 既存のブランドデータを使うかどうか
# brand_output_path : 抽出したブランドリストを出力するパス

data_model = data_preprocessing_operator(start_date="2020/01/01 00:00:00",
                                        end_date="2024/06/15 23:59:59",
                                        openai_api_key=config['openai']['api_key'],
                                        use_existing_brand=True,
                                        brand_output_path='brand_lists/brand_list_20200101_20240616.csv')

existing_brand_path = 'brand_lists/brand_list_20200101_20240616.csv'

df = data_model.process( 
                df_frame,
                df_katashiki,
                df_pachinko,
                df_slot,
                df_katashiki_frame_relation,
                df_model_evaluation,
                df_pachinko_type,
                df_pachinko_type_detail, existing_brand_path)

In [7]:
import pandas as pd
from src.modeling import modeling
from src.evaluation import evaluation

cols = df.columns.tolist()

targets = [
     'Performance',
     'Spec',
     'Content',
     'News',
     'Concept',
     'Period',
     'SaleUnit_eval',
     'Running',
     'Returns',
     'Price_eval']

features = [col for col in cols if col not in targets + ['Name', 'ModelLife', 'GrossProfit', 'KatashikiId', 'Regist_Date', 'Color', 'TypeName', 'TypeName_detail', 'brand_name']]

# 学習とテストデータの定義
df_train = df[df['Regist_Date'] < df['Regist_Date'].min() + pd.DateOffset(days=365*4 - 90)]
df_test = df[df['Regist_Date'] >= df['Regist_Date'].min() + pd.DateOffset(days=365*4 - 90)]
print(df_test['Regist_Date'].min(), df_test['Regist_Date'].max())
    
# 学習開始
modeling = modeling(features, targets)
models, df_output = modeling.train_predict_all(df_train, df_test)

df_output = get_actual_pred_abs_diff(df_output, features_en_to_jp)

features_2 = [x for x in features if x not in ['KatashikiId', 'Price', 'Name', 'brand_name']]
df_eval = pd.concat([df_test[['KatashikiId', 'Price', 'Name', 'brand_name']], df_output, df_test[features_2]], axis=1) 

df_eval = df_eval.rename(columns=features_en_to_jp)

2023-10-23 18:37:01 2024-06-21 10:05:44


In [8]:
df_eval = df_eval.rename(columns={'KatashikiId':'型式ID', 'Name': '型式名', 'brand_name':'ブランド名（型式名から抽出）'})

In [9]:
df_eval.to_csv('output_analysis/実績値と予測値の比較_20231023_20240621.csv', index=False)

# 推論20240622~20240806

In [10]:
############### 最新のデータを読み込む ###############
from src.data_preprocessing import data_preprocessing_operator
import configparser
import pandas as pd
config = configparser.ConfigParser()
config.read('config.ini')

# start_date :　抽出データ開始日
# end_date : 抽出データ終了日
# openai_api_key : openaiのAPIキー
# use_existing_brand : 既存のブランドデータを使うかどうか
# brand_output_path : 抽出したブランドリストを出力するパス

data_model = data_preprocessing_operator(start_date="2020/01/01 00:00:00",
                                        end_date="2024/08/06 23:59:59",
                                        openai_api_key=config['openai']['api_key'],
                                        use_existing_brand=True,
                                        brand_output_path='brand_lists/brand_list_20200101_20240728.csv', # 新しいアウトプットパス
                                        fit_categorical_values=False)

existing_brand_path = 'brand_lists/brand_list_20200101_20240616.csv'

df = data_model.process( 
                df_frame,
                df_katashiki,
                df_pachinko,
                df_slot,
                df_katashiki_frame_relation,
                df_model_evaluation,
                df_pachinko_type,
                df_pachinko_type_detail, existing_brand_path)

# ここで特定の日付以上にする
df = df[df['Regist_Date'] >= "2024/06/22 00:00:00"]

############### モデルの読み込み ###############
# run_jobs.ipynbで保存したモデル
folder = 'output_train/train20200117to20231017_test20231023to20240621'
suffix = 'rf_default'
import pickle
with open(folder + '/model_'+suffix+'.pickle', 'rb') as handle:
    models = pickle.load(handle)

############### 推論　###############
from src.modeling import modeling

cols = df.columns.tolist()

targets = [
     'Performance',
     'Spec',
     'Content',
     'News',
     'Concept',
     'Period',
     'SaleUnit_eval',
     'Running',
     'Returns',
     'Price_eval']

features = [col for col in cols if col not in targets + ['Name', 'ModelLife', 'GrossProfit', 'KatashikiId', 'Regist_Date', 'Color', 'TypeName', 'TypeName_detail', 'brand_name']]

modeling = modeling(features, targets)
df_output = modeling.predict_all(models, df)

df_output = get_actual_pred_abs_diff(df_output, features_en_to_jp)

features_2 = [x for x in features if x not in ['KatashikiId', 'Price', 'Name', 'brand_name']]
df_eval_2 = pd.concat([df[['KatashikiId', 'Price', 'Name', 'brand_name']], df_output, df[features_2]], axis=1) 

df_eval_2 = df_eval_2.rename(columns=features_en_to_jp)

In [11]:
df_eval_2 = df_eval_2.rename(columns={'KatashikiId':'型式ID', 'Name': '型式名', 'brand_name':'ブランド名（型式名から抽出）'})

In [12]:
df_eval_2.to_csv('output_analysis/実績値と予測値の比較_20240622_20240806.csv', index=False)